**Wilbeibi's Pyspark Note**
=======

Document: [pyspark package](https://spark.apache.org/docs/latest/api/python/pyspark.html)

### Basics: generate RDD and `map`
    1. `sc.parallelize` to generate RDD, which typed as `<class 'pyspark.rdd.RDD'>`.
    2. use RDD's `map` to operate each element in RDD, and `collect` to show the results.
    3. list to RDD becomes rdd, generator (map function and so forth) to RDD becomes pipelinedRDD.

In [10]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
range_vals = xrange(200)

wordsRDD = sc.parallelize(wordsList, 4)
rangeRDD = sc.parallelize(range_vals, 20)
# Print out the type of wordsRDD
print ("wordsRDD's type: ", type(wordsRDD))
print ("rangeRDD's type: ", type(rangeRDD))
pluralLambdaRDD = wordsRDD.map(lambda w: w + 's')
print pluralLambdaRDD.collect(), type(pluralLambdaRDD)

("wordsRDD's type: ", <class 'pyspark.rdd.RDD'>)
("rangeRDD's type: ", <class 'pyspark.rdd.PipelinedRDD'>)
['cats', 'elephants', 'rats', 'rats', 'cats'] <class 'pyspark.rdd.PipelinedRDD'>


### `groupByKey() approach`
groupByKey() transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. When sort the RDD, we might need to use it

In [6]:
# map word RDD into word:value pairs
wordPairs = wordsRDD.map(lambda w:(w, 1))
print wordPairs.collect()
# group by word
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print '{0}: {1}'.format(key, list(value))
# count by words
wordCountsGrouped = wordsGrouped.map(lambda (k, v): (k, sum(v)))
print wordCountsGrouped.collect()

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]
rat: [1, 1]
elephant: [1]
cat: [1, 1]
[('rat', 2), ('elephant', 1), ('cat', 2)]


### `reduceByKey() approach`
The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

In [8]:
wordCounts = wordPairs.reduceByKey(lambda a, b: a+b)
print wordCounts.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


### More operations on RDD
+ `reduce()` works like python `reduce()`, the only differece is that it works on each element in RDD.
+ `take(count)`, take first *count* elements. 
+ `filter(func)`, remove the elements which are `False` under *func*.
+ `takeOrdered(count, func)`, take *count* elements by *func*.

### A little bit regex
+ Reference: [Python Regular Expressions: Essentional Materials](https://developers.google.com/edu/python/regular-expressions)
+ `match = re.search(pat, str)` to search patterns
+ `re.group(idx)` to show the `idx`th match

### RDD Creation
+ `sc.textfile(logFile)` to convert each line of the file into an element in an RDD.
+ `.cache()` operation in RDD Object will cache the data in memory. Needed if we want to seach the same results in further.